# 📊 Credit Risk Analysis

This project aims to predict credit risk by determining which customers are likely to pay their loans on time and which are not. We'll employ various data science techniques and methodologies.

## 🎯 Objectives

We will cover the following concepts:

1. **🔍 Exploratory Data Analysis (EDA)**:
   - Analyze the dataset to uncover patterns, spot anomalies, and check assumptions using statistical summaries and visualizations.

2. **🛠️ Data Preprocessing**:
   - Clean the data, handle missing values, encode categorical variables, normalize data, and split into training and testing sets.

3. **⭐ Feature Importance**:
   - Identify key features influencing `loan_status` to improve model performance and interpretability.

4. **🔽 Dimensionality Reduction**:
   - Use techniques like PCA to reduce the number of features while retaining essential information, speeding up model training, and reducing overfitting.

5. **🤖 Predictive Modeling**:
   - Build various models (logistic regression, decision trees, random forests, gradient boosting) to predict the target variable `loan_status`.

6. **⚙️ Hyperparameter Optimization**:
   - Perform hyperparameter tuning with Optuna to find the best parameters for our models.

7. **🧪 Model Testing**:
   - Evaluate models using metrics like accuracy, precision, recall, F1 score, and AUC-ROC to test performance on unseen data.

## 🎯 Goal

Predict the `loan_status` to determine the likelihood of customers paying their loans on time. This helps financial institutions make informed loan approval decisions and manage risk effectively.


# 📖 Data Dictionary

| Column Name            | Description                                                                                           |
|------------------------|-------------------------------------------------------------------------------------------------------|
| out_prncp_inv          | Remaining outstanding principal for portion of total amount funded by investors                       |
| policy_code            | Publicly available policy_code=1. <br> New products not publicly available policy_code=2                   |
| pub_rec                | Number of derogatory public records                                                                   |
| purpose                | A category provided by the borrower for the loan request.                                             |
| pymnt_plan             | Indicates if a payment plan has been put in place for the loan                                        |
| recoveries             | Post charge off gross recovery                                                                        |
| revol_bal              | Total credit revolving balance                                                                        |
| revol_util             | Revolving line utilization rate, or the amount of credit the borrower is using relative to all available revolving credit. |
| sub_grade              | LC assigned loan subgrade                                                                             |
| term                   | The number of payments on the loan. Values are in months and can be either 36 or 60.                  |
| title                  | The loan title provided by the borrower                                                               |
| total_acc              | The total number of credit lines currently in the borrower's credit file                              |
| total_pymnt            | Payments received to date for total amount funded                                                     |
| total_pymnt_inv        | Payments received to date for portion of total amount funded by investors                             |
| total_rec_int          | Interest received to date                                                                             |
| total_rec_late_fee     | Late fees received to date                                                                            |
| total_rec_prncp        | Principal received to date                                                                            |
| url                    | URL for the LC page with listing data.                                                                |
| verified_status_joint  | Indicates if the co-borrowers' joint income was verified by LC, not verified, or if the income source was verified |
| zip_code               | The first 3 numbers of the zip code provided by the borrower in the loan application.                 |
| open_acc_6m            | Number of open trades in last 6 months                                                                |
| open_il_6m             | Number of currently active installment trades                                                         |
| open_il_12m            | Number of installment accounts opened in past 12 months                                               |
| open_il_24m            | Number of installment accounts opened in past 24 months                                               |
| mths_since_rcnt_il     | Months since most recent installment accounts opened                                                  |
| total_bal_il           | Total current balance of all installment accounts                                                     |
| il_util                | Ratio of total current balance to high credit/credit limit on all install acct                        |
| open_rv_12m            | Number of revolving trades opened in past 12 months                                                   |
| open_rv_24m            | Number of revolving trades opened in past 24 months                                                   |
| max_bal_bc             | Maximum current balance owed on all revolving accounts                                                |
| all_util               | Balance to credit limit on all trades                                                                 |
| total_rev_hi_lim       | Total revolving high credit/credit limit                                                              |
| inq_fi                 | Number of personal finance inquiries                                                                  |
| total_cu_tl            | Number of finance trades                                                                              |
| inq_last_12m           | Number of credit inquiries in past 12 months                                                          |
| acc_now_delinq         | The number of accounts on which the borrower is now delinquent.                                       |
| tot_coll_amt           | Total collection amounts ever owed                                                                    |
| tot_cur_bal            | Total current balance of all accounts                                                                 |

#### * Employer Title replaces Employer Name for all loans listed after 9/23/2013



## Some important features

| Column      | Description                                  |
|-------------|----------------------------------------------|
| loan_amnt   | Amount of money requested by the borrower.   |
| int_rate    | Interest rate of the loan.                   |
| grade       | Loan grade with categories A, B, C, D, E, F, G. |
| annual_inc  | Borrower's annual income.                    |
| purpose     | The primary purpose of borrowing.            |
| installment | Monthly amount payments for opted loan.      |
| term        | Duration of the loan until it’s paid off.    |


# 🚀 Importing libraries and getting started

In [2]:
import pandas as pd
import numpy as np

import plotly.express as px
from sklearn import metrics

In [3]:
data = pd.read_csv('C:/Users/saran/OneDrive/Documents/GitHub/credit-risk/data/loan.csv', low_memory=False)

In [4]:
print(data.shape)
print(data.shape[0] * data.shape[1])

(887379, 74)
65666046


### We have about 887K rows and 74 columns, which amounts to about 65 million data points.

In [5]:
data['loan_status'].unique() # Understanding the target variable

array(['Fully Paid', 'Charged Off', 'Current', 'Default',
       'Late (31-120 days)', 'In Grace Period', 'Late (16-30 days)',
       'Does not meet the credit policy. Status:Fully Paid',
       'Does not meet the credit policy. Status:Charged Off', 'Issued'],
      dtype=object)

### Here is what the terms in the target variable mean
| Term                                               | Meaning                                                                                                                                                 |
|----------------------------------------------------|---------------------------------------------------------------------------------------------------------------------------------------------------------|
| **Fully Paid**                                     | The borrower has completely repaid the loan.                                                                                                             |
| **Charged Off**                                    | The lender has given up on collecting the loan because the borrower hasn’t paid for a long time. The loan is considered a loss.                          |
| **Current**                                        | The borrower is making payments on time.                                                                                                                 |
| **Default**                                        | The borrower has stopped making payments for a long period, and the loan is in serious trouble.                                                          |
| **Late (31-120 days)**                             | The borrower has missed payments and is behind by 31 to 120 days.                                                                                        |
| **In Grace Period**                                | The borrower missed a payment, but the late fee hasn’t been applied yet because it’s within an allowed period after the due date.                        |
| **Late (16-30 days)**                              | The borrower is behind on payments by 16 to 30 days.                                                                                                     |
| **Does not meet the credit policy. Status: Fully Paid** | The loan didn’t meet the lender's usual criteria but was still given and has been completely repaid.                                                      |
| **Does not meet the credit policy. Status: Charged Off** | The loan didn’t meet the lender's usual criteria, was still given, but ended up in loss as the borrower didn’t repay.                                      |
| **Issued**                                         | The loan has been approved and the money has been given to the borrower.                                                                                 |


In [6]:
for i in data.columns[2:]:
    print(i)
    print(data[i].head(), '\n')

loan_amnt
0     5000.0
1     2500.0
2     2400.0
3    10000.0
4     3000.0
Name: loan_amnt, dtype: float64 

funded_amnt
0     5000.0
1     2500.0
2     2400.0
3    10000.0
4     3000.0
Name: funded_amnt, dtype: float64 

funded_amnt_inv
0     4975.0
1     2500.0
2     2400.0
3    10000.0
4     3000.0
Name: funded_amnt_inv, dtype: float64 

term
0     36 months
1     60 months
2     36 months
3     36 months
4     60 months
Name: term, dtype: object 

int_rate
0    10.65
1    15.27
2    15.96
3    13.49
4    12.69
Name: int_rate, dtype: float64 

installment
0    162.87
1     59.83
2     84.33
3    339.31
4     67.79
Name: installment, dtype: float64 

grade
0    B
1    C
2    C
3    C
4    B
Name: grade, dtype: object 

sub_grade
0    B2
1    C4
2    C5
3    C1
4    B5
Name: sub_grade, dtype: object 

emp_title
0                         NaN
1                       Ryder
2                         NaN
3         AIR RESOURCES BOARD
4    University Medical Group
Name: emp_title, dtype: ob